In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Rescaling

# Direktori dataset
dataset_dir = r"C:\Users\ACER\Downloads\Project Capstone\data"  # Sesuaikan dengan path dataset Anda

# Membagi dataset menjadi training (80%) dan testing (20%)
train_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(128, 128),  # Ukuran gambar disesuaikan dengan model
    batch_size=32
)

test_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(128, 128),  # Ukuran gambar disesuaikan dengan model
    batch_size=32
)

# Normalisasi dataset (mengubah nilai piksel menjadi 0-1)
normalization_layer = Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Prefetch data untuk efisiensi
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

# 1. Memuat model pretrained MobileNetV2
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

# Jangan latih layer dasar (base layers)
base_model.trainable = False

# 2. Membuat model baru dengan MobileNetV2 sebagai base
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Output sigmoid untuk klasifikasi biner
])

# 3. Compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 4. Melatih model
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

# Menyimpan model
model.save("fish_freshness_model_v2.h5")

print("Model telah selesai dilatih dan disimpan.")


Found 4876 files belonging to 2 classes.
Using 3901 files for training.
Found 4876 files belonging to 2 classes.
Using 975 files for validation.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 8s 1us/step
Epoch 1/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 36s 258ms/step - accuracy: 0.9282 - loss: 0.1647 - val_accuracy: 0.9969 - val_loss: 0.0096
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 29s 240ms/step - accuracy: 0.9989 - loss: 0.0091 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 3/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 29s 237ms/step - accuracy: 0.9995 - loss: 0.0035 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 4/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 29s 238ms/step - accuracy: 0.9995 - loss: 0.0024 - val_accuracy: 1.0000 - val_loss: 8.5869e-04
Epoch 5/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 29s 238ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.9969 - val_loss: 0.0081
Epoch 6/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 29s 237ms/step - accuracy: 0.9987 - loss: 0.0030 - val_accuracy: 1.0000 - val_loss: 5.3026e-04
Epoch 7/10
1

Model telah selesai dilatih dan disimpan.
